# KDDCUP Data Extract

2017-05-11

### 录入数据 

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date,time
import copy

In [2]:
# 读入平均时间，流量和天气数据（平均时间和流量由aggregate_travel_time.py和aggregate_volume.py生成）
in_file_path = '/home/godcedric/GitLocal/KDDCUP2017/result/training_20min_avg_travel_time.csv'
raw_data = pd.read_csv(in_file_path)
in_file_path = '/home/godcedric/GitLocal/KDDCUP2017/result/training_20min_avg_volume.csv'
raw_data2 = pd.read_csv(in_file_path)
in_file_path = '/home/godcedric/GitLocal/KDDCUP2017/result/weather (table 7)_training_update.csv'
weather_data = pd.read_csv(in_file_path)

In [3]:
# 训练集缺失值补充，保证每一个时间段都有数据
full_time = pd.read_csv('full_time.csv')
full_time = full_time.sort(['intersection_id','tollgate_id'])
raw_data = pd.merge(full_time, raw_data, on=['intersection_id','tollgate_id','time_window'], how='left')
ffill = lambda g:g.fillna(method='ffill')
bfill = lambda g:g.fillna(method='bfill')

raw_data['avg_travel_time'] = raw_data.groupby(['intersection_id','tollgate_id'])['avg_travel_time'].apply(ffill)
raw_data['avg_travel_time'] = raw_data.groupby(['intersection_id','tollgate_id'])['avg_travel_time'].apply(bfill)

full_time2 = pd.read_csv('full_time2.csv')
full_time2 = full_time2.sort(['tollgate_id','direction'])
raw_data2 = pd.merge(full_time2, raw_data2, on=['tollgate_id','time_window','direction'], how='left')
raw_data2['volume'] = raw_data2.groupby(['tollgate_id','direction'])['volume'].apply(ffill)
raw_data2['volume'] = raw_data2.groupby(['tollgate_id','direction'])['volume'].apply(bfill)

/home/godcedric/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
/home/godcedric/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [4]:
raw_data2

,tollgate_id,time_window,direction,volume,etc
0,1,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",0,13.0,4.0
1,1,"[2016-09-19 00:20:00,2016-09-19 00:40:00)",0,6.0,1.0
2,1,"[2016-09-19 00:40:00,2016-09-19 01:00:00)",0,9.0,1.0
3,1,"[2016-09-19 01:00:00,2016-09-19 01:20:00)",0,10.0,1.0
4,1,"[2016-09-19 01:20:00,2016-09-19 01:40:00)",0,14.0,3.0
5,1,"[2016-09-19 01:40:00,2016-09-19 02:00:00)",0,10.0,0.0
6,1,"[2016-09-19 02:00:00,2016-09-19 02:20:00)",0,7.0,1.0
7,1,"[2016-09-19 02:20:00,2016-09-19 02:40:00)",0,10.0,1.0
8,1,"[2016-09-19 02:40:00,2016-09-19 03:00:00)",0,6.0,0.0
9,1,"[2016-09-19 03:00:00,2016-09-19 03:20:00)",0,9.0,0.0


### 提取平均时间数据 

In [5]:
# 整合平均时间与天气数据
raw_data['start_time'] = raw_data['time_window'].map(lambda x: datetime.strptime(x.split(',')[0][1:], '%Y-%m-%d %H:%M:%S'))
weather_data['date'] = weather_data['date'].astype(str)
weather_data['hour'] = weather_data['hour'].map(lambda x: ' ' + time(x, 0, 0).strftime('%H:%M:%S'))
weather_data['start_time'] = weather_data['date'] + weather_data['hour']
weather_data['start_time'] = weather_data['start_time'].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
num = len(weather_data)
for i in range(num):
    temp = weather_data.ix[i]
    temp1 = copy.deepcopy(temp)
    temp2 = copy.deepcopy(temp)
    temp3 = copy.deepcopy(temp)
    temp4 = copy.deepcopy(temp)
    temp5 = copy.deepcopy(temp)
    temp6 = copy.deepcopy(temp)
    temp7 = copy.deepcopy(temp)
    temp8 = copy.deepcopy(temp)
    stime = temp.start_time
    temp1.start_time = stime + timedelta(minutes=20)
    temp2.start_time = stime + timedelta(minutes=40)
    temp3.start_time = stime + timedelta(minutes=60)
    temp4.start_time = stime + timedelta(minutes=80)
    temp5.start_time = stime + timedelta(minutes=100)
    temp6.start_time = stime + timedelta(minutes=120)
    temp7.start_time = stime + timedelta(minutes=140)
    temp8.start_time = stime + timedelta(minutes=160)
    alltemp = [temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8]
    alltemp = pd.DataFrame(alltemp)
    weather_data = pd.concat([weather_data, alltemp])
weather_data.sort('start_time')

process_data = pd.merge(raw_data, weather_data, on='start_time', how='left')
del process_data['hour']
process_data['time'] = process_data['start_time'].map(lambda x: x.time())

/home/godcedric/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:30: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [6]:
# 增加路线和星期几特征
process_data['weekday'] = process_data['start_time'].map(lambda x: x.weekday())
process_data['route'] = process_data['intersection_id'].astype(str) + '-' + process_data['tollgate_id'].astype(str)

In [7]:
process_data

,intersection_id,tollgate_id,time_window,avg_travel_time,start_time,date,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,time,weekday,route
0,A,2,"[2016-07-19 00:00:00,2016-07-19 00:20:00)",58.05,2016-07-19 00:00:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,00:00:00,1,A-2
1,A,2,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",58.05,2016-07-19 00:20:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,00:20:00,1,A-2
2,A,2,"[2016-07-19 00:40:00,2016-07-19 01:00:00)",58.05,2016-07-19 00:40:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,00:40:00,1,A-2
3,A,2,"[2016-07-19 01:00:00,2016-07-19 01:20:00)",58.05,2016-07-19 01:00:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,01:00:00,1,A-2
4,A,2,"[2016-07-19 01:20:00,2016-07-19 01:40:00)",56.87,2016-07-19 01:20:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,01:20:00,1,A-2
5,A,2,"[2016-07-19 01:40:00,2016-07-19 02:00:00)",77.74,2016-07-19 01:40:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,01:40:00,1,A-2
6,A,2,"[2016-07-19 02:00:00,2016-07-19 02:20:00)",77.74,2016-07-19 02:00:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,02:00:00,1,A-2
7,A,2,"[2016-07-19 02:20:00,2016-07-19 02:40:00)",42.64,2016-07-19 02:20:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,02:20:00,1,A-2
8,A,2,"[2016-07-19 02:40:00,2016-07-19 03:00:00)",40.17,2016-07-19 02:40:00,2016-07-19,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,02:40:00,1,A-2
9,A,2,"[2016-07-19 03:00:00,2016-07-19 03:20:00)",40.17,2016-07-19 03:00:00,2016-07-19,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,03:00:00,1,A-2


In [9]:
# 增加前20分钟平均时间特征
start_time = process_data['start_time']
avg_travel_time = process_data['avg_travel_time']

In [6]:
last_20min = process_data['avg_travel_time'].copy() 
last_40min = process_data['avg_travel_time'].copy()
last_60min = process_data['avg_travel_time'].copy()
last_80min = process_data['avg_travel_time'].copy()
last_100min = process_data['avg_travel_time'].copy()
last_120min = process_data['avg_travel_time'].copy()
last_20min[:] = np.nan
last_40min[:] = np.nan
last_60min[:] = np.nan
last_80min[:] = np.nan
last_100min[:] = np.nan
last_120min[:] = np.nan
for i in range(len(start_time)):
    cur_time = start_time[i] #当前时间
    # 前20min
    if (i-1) >= 0:
        last20 = cur_time - timedelta(minutes=20)
        if last20 == start_time[i-1]:
            last_20min[i] = avg_travel_time[i-1]
    # 前40min
    if (i-2) >= 0:
        last40 = cur_time - timedelta(minutes=40)
        for j in range(2):
            if last40 == start_time[i-j-1]:
                last_40min[i] = avg_travel_time[i-j-1]
                break
    # 前60min
    if (i-3) >= 0:
        last60 = cur_time - timedelta(minutes=60)
        for j in range(3):
            if last60 == start_time[i-j-1]:
                last_60min[i] = avg_travel_time[i-j-1]
                break
    # 前80min
    if (i-4) >= 0:
        last80 = cur_time - timedelta(minutes=80)
        for j in range(4):
            if last80 == start_time[i-j-1]:
                last_80min[i] = avg_travel_time[i-j-1]
                break
    # 前100min
    if (i-5) >= 0:
        last100 = cur_time - timedelta(minutes=100)
        for j in range(5):
            if last100 == start_time[i-j-1]:
                last_100min[i] = avg_travel_time[i-j-1]
                break
    # 前120min
    if (i-6) >= 0:
        last120 = cur_time - timedelta(minutes=120)
        for j in range(6):
            if last120 == start_time[i-j-1]:
                last_120min[i] = avg_travel_time[i-j-1]
                break

process_data['last_20min'] = last_20min
process_data['last_40min'] = last_40min
process_data['last_60min'] = last_60min
process_data['last_80min'] = last_80min
process_data['last_100min'] = last_100min
process_data['last_120min'] = last_120min

In [8]:
# 把time映射成1～72
from collections import defaultdict
time_start = datetime(2016,10,17,0,0,0)
timedic = defaultdict(int)
for i in range(72):
    timedic[time_start.time()] = i+1
    time_start = time_start + timedelta(minutes=20) 

In [9]:
process_data['timemap'] = process_data['time'].map(lambda x:timedic[x])

In [10]:
# 增加法定节假日特征
holiday = ['2016-09-15','2016-09-16','2016-09-17','2016-10-01','2016-10-02','2016-10-03','2016-10-04','2016-10-05','2016-10-06','2016-10-07']
def ff(x):
    if x in holiday:
        return 1
    else:
        return 0
process_data['holiday'] = process_data['date'].map(ff)

In [14]:
# 把速度列加上去
#avg_speed = pd.read_csv('avg_speed_all_route_and_last_link.csv')
#process_data['avg_all_route_speed'] = list(avg_speed['avg_all_route_speed'])
#process_data['avg_last_link_speed'] = list(avg_speed['avg_last_link_speed'])

In [11]:
# 写出数据
process_data.to_csv('travel_time_raw_data_ffill.csv', index=False)

### 提取流量数据 

In [12]:
# 整合流量与天气数据
raw_data2['start_time'] = raw_data2['time_window'].map(lambda x: datetime.strptime(x.split(',')[0][1:],'%Y-%m-%d %H:%M:%S'))
raw_data2['pair'] = raw_data2['tollgate_id'].astype(str) + '-' + raw_data2['direction'].astype(str)
process_data2 = pd.merge(raw_data2, weather_data, on='start_time', how='left')

In [13]:
# 增加星期几和时间窗口两列
process_data2['weekday'] = process_data2['start_time'].map(lambda x: x.weekday())
process_data2['time'] = process_data2['start_time'].map(lambda x: x.time())

In [21]:
# 增加前20分钟流量特征
start_time = process_data2['start_time']
volume = process_data2['volume']

In [23]:
last_20min = process_data2['volume'].copy() 
last_40min = process_data2['volume'].copy()
last_60min = process_data2['volume'].copy()
last_80min = process_data2['volume'].copy()
last_100min = process_data2['volume'].copy()
last_120min = process_data2['volume'].copy()
last_20min[:] = np.nan
last_40min[:] = np.nan
last_60min[:] = np.nan
last_80min[:] = np.nan
last_100min[:] = np.nan
last_120min[:] = np.nan
for i in range(len(start_time)):
    cur_time = start_time[i] #当前时间
    # 前20min
    if (i-1) >= 0:
        last20 = cur_time - timedelta(minutes=20)
        if last20 == start_time[i-1]:
            last_20min[i] = volume[i-1]
    # 前40min
    if (i-2) >= 0:
        last40 = cur_time - timedelta(minutes=40)
        for j in range(2):
            if last40 == start_time[i-j-1]:
                last_40min[i] = volume[i-j-1]
                break
    # 前60min
    if (i-3) >= 0:
        last60 = cur_time - timedelta(minutes=60)
        for j in range(3):
            if last60 == start_time[i-j-1]:
                last_60min[i] = volume[i-j-1]
                break
    # 前80min
    if (i-4) >= 0:
        last80 = cur_time - timedelta(minutes=80)
        for j in range(4):
            if last80 == start_time[i-j-1]:
                last_80min[i] = volume[i-j-1]
                break
    # 前100min
    if (i-5) >= 0:
        last100 = cur_time - timedelta(minutes=100)
        for j in range(5):
            if last100 == start_time[i-j-1]:
                last_100min[i] = volume[i-j-1]
                break
    # 前120min
    if (i-6) >= 0:
        last120 = cur_time - timedelta(minutes=120)
        for j in range(6):
            if last120 == start_time[i-j-1]:
                last_120min[i] = volume[i-j-1]
                break

process_data2['last_20min'] = last_20min
process_data2['last_40min'] = last_40min
process_data2['last_60min'] = last_60min
process_data2['last_80min'] = last_80min
process_data2['last_100min'] = last_100min
process_data2['last_120min'] = last_120min

In [14]:
process_data2

,tollgate_id,time_window,direction,volume,etc,start_time,pair,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,weekday,time
0,1,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",0,13.0,4.0,2016-09-19 00:00:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,00:00:00
1,1,"[2016-09-19 00:20:00,2016-09-19 00:40:00)",0,6.0,1.0,2016-09-19 00:20:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,00:20:00
2,1,"[2016-09-19 00:40:00,2016-09-19 01:00:00)",0,9.0,1.0,2016-09-19 00:40:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,00:40:00
3,1,"[2016-09-19 01:00:00,2016-09-19 01:20:00)",0,10.0,1.0,2016-09-19 01:00:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,01:00:00
4,1,"[2016-09-19 01:20:00,2016-09-19 01:40:00)",0,14.0,3.0,2016-09-19 01:20:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,01:20:00
5,1,"[2016-09-19 01:40:00,2016-09-19 02:00:00)",0,10.0,0.0,2016-09-19 01:40:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,01:40:00
6,1,"[2016-09-19 02:00:00,2016-09-19 02:20:00)",0,7.0,1.0,2016-09-19 02:00:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,02:00:00
7,1,"[2016-09-19 02:20:00,2016-09-19 02:40:00)",0,10.0,1.0,2016-09-19 02:20:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,02:20:00
8,1,"[2016-09-19 02:40:00,2016-09-19 03:00:00)",0,6.0,0.0,2016-09-19 02:40:00,1-0,2016-09-19,00:00:00,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0,0,02:40:00
9,1,"[2016-09-19 03:00:00,2016-09-19 03:20:00)",0,9.0,0.0,2016-09-19 03:00:00,1-0,2016-09-19,03:00:00,1008.3,1013.3,15.0,4.5,24.4,67.0,0.0,0,03:00:00


In [15]:
# 时间映射成1～72
process_data2['timemap'] = process_data2['time'].map(lambda x:timedic[x])

In [16]:
process_data2['holiday'] = process_data2['date'].map(ff)

In [17]:
# 写出数据
process_data2.to_csv('volume_raw_data_ffill.csv', index=False)